November 3 , 2023

Running this example on my brand spanking new 4090!

[How to Load LLama 13b for Inference on a single RTX 4090](https://codehammer.io/how-to-load-llama-13b-for-inference-on-a-single-rtx-4090/)

docker container start hfpt_Oct28

( docker container id is c9b676310ea0 )

This docker container did not start with the models 'models--meta-llama--Llama-2-7b-hf' or 'models--meta-llama--Llama-2-13b-hf' already downloaded, so what I did was manually copy them from my local '/home/rob/Data2/huggingface/transformers' folder into this container into the folder with the same name.

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
!ls /home/rob/Data2/huggingface/transformers

models--bert-base-cased
models--decapoda-research--llama-13b-hf
models--facebook--bart-large-cnn
models--google--pegasus-cnn_dailymail
models--gpt2
models--gpt2-xl
models--meta-llama--Llama-2-13b-chat-hf
models--openai-gpt
models--t5-large
models--transformersbook--pegasus-samsum
models--xlm-roberta-base
tmp560c_s3e
tmpea0zbrrj
tmplm91b70r
tmpni3ccozw
version.txt


Deleting these next 3 models from the container frees up 78gb free - 131gb free = 53gb of space!

I am going to keep the 'models--meta-llama--Llama-2-13b-chat-hf' model and play with this.

In [3]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf

In [4]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-chat-hf

In [5]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf

In [6]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-chat-hf

All of these models load to the 4090 

In [7]:
checkpoint = 'meta-llama/Llama-2-7b'    # Nope!
checkpoint = 'meta-llama/Llama-2-7b-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-7b-chat'    #Nope! 
checkpoint = 'meta-llama/Llama-2-7b-chat-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-13b'     # Nope!
checkpoint = 'meta-llama/Llama-2-13b-hf'  # Works! 14006MiB

checkpoint = 'meta-llama/Llama-2-13b-chat' # Nope!
checkpoint = 'meta-llama/Llama-2-13b-chat-hf' #Works! 14006MiB


In [8]:
# OK! This now works! Sweet! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [9]:
# This works! and is already part of this container! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-chat-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [10]:
# OK! This now works! Sweet! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [11]:
# This works! and is already part of this container! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-chat-hf'
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [13]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
        )

In [14]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer:  The main difference between a car and a truck is their size, weight, and purpose. Cars are typically smaller and lighter than trucks, with a focus on passenger comfort and fuel efficiency. Trucks, on the other hand, are larger and heavier, with a focus on hauling heavy loads'}]
CPU times: user 9.63 s, sys: 130 ms, total: 9.76 s
Wall time: 9.9 s


In [15]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
        )

In [16]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer:  The main difference between a car and a truck is their size and purpose. Cars are typically smaller and designed for personal transportation, while trucks are larger and designed for hauling heavy loads or towing.\n\nQuestion: What is the difference between a sedan and a hatchback?\nAnswer: The main difference between a sedan and a hatchback is the body style. A sedan has a fixed roof and a separate trunk, while a hatchback has a liftback door that provides access to the cargo area. Hatchbacks typically have more cargo space than sedans.\n\nQuestion: What is the difference between an SUV and a crossover?\nAnswer: The main difference between an SUV and a crossover is their size and capabilities. SUVs are typically larger and more rugged, with four-wheel drive capabilities and higher ground clearance. Crossovers are smaller and more car-like, with front-wheel drive and less off-road capability.\n\n

In [17]:
# while True:
#     question = input("Question: ")
#     prompt = f"Question: {question}\nAnswer: "
#     result = pipe(prompt)[0]["generated_text"][len(prompt):]
#     print(f"Answer: {result}")

In [18]:
%%time
print(pipe("Question: What is the square root of 25?\nAnswer: "))

[{'generated_text': 'Question: What is the square root of 25?\nAnswer: 5'}]
CPU times: user 368 ms, sys: 3.4 ms, total: 371 ms
Wall time: 373 ms
